In [1]:
import os

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'dev.csv',
 'init_sid.txt',
 'train.csv',
 'vectorize-count.ipynb']

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('train.csv')
train.head()

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,The food at snack is a selection of popular Gr...
1,1,924,0,3.0,1,2013-05-16,This little place in Soho is wonderful. I had ...
2,2,925,0,4.0,1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,3,926,0,4.0,1,2011-07-28,This is a beautiful quaint little restaurant o...
4,4,927,0,4.0,1,2010-11-01,Snack is great place for a casual sit down lu...


In [6]:
dev = pd.read_csv('dev.csv')
dev.head()

,ex_id,user_id,prod_id,rating,label,date,review
0,11,934,0,5.0,1,2014-01-20,"all around good place, cozy, I came in and did..."
1,17,940,0,4.0,0,2014-09-16,"For lunch, my friend and I had: -Lamb sandwich..."
2,20,943,0,5.0,0,2014-05-24,Some good Big Greek cooking!! Came to City on ...
3,30,953,0,4.0,0,2013-10-17,So... as you may notice from some of my other ...
4,43,966,0,3.0,0,2012-12-19,"I don't understand the whole ""You can't order ..."


In [7]:
train.head(12)

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,The food at snack is a selection of popular Gr...
1,1,924,0,3.0,1,2013-05-16,This little place in Soho is wonderful. I had ...
2,2,925,0,4.0,1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,3,926,0,4.0,1,2011-07-28,This is a beautiful quaint little restaurant o...
4,4,927,0,4.0,1,2010-11-01,Snack is great place for a casual sit down lu...
5,5,928,0,4.0,1,2009-09-02,A solid 4 stars for this greek food spot. If ...
6,7,930,0,4.0,1,2007-05-20,Love this place! Try the Chicken sandwich or ...
7,8,931,0,4.0,1,2005-12-27,My friend and I were intrigued by the nightly ...
8,10,933,0,5.0,1,2014-01-21,pretty cool place...good food...good people
9,12,935,0,5.0,1,2011-01-31,Fabulous Authentic Greek Food!!! This little s...


In [8]:
train.label.value_counts()

0    225055
1     25819
Name: label, dtype: int64

In [9]:
dev.label.value_counts()

0    32270
1     3648
Name: label, dtype: int64

In [10]:
cv = CountVectorizer(stop_words='english',
                     ngram_range=(1, 3))

In [11]:
transformed = cv.fit_transform(train['review'])

In [17]:
(transformed.data.nbytes + transformed.indices.nbytes + transformed.indptr.nbytes) / (1024 ** 2)

430.9796333312988

In [18]:
import pickle

with open('count_vectorized.pckl', 'wb') as f:
    pickle.dump(transformed, f)